# 최저기온 

POI별 관광객을 예측하기 위한 피쳐로써 날씨 데이터를 활용 
이 코드에서는 최저 기온의 전처리를 진행함

[데이터셋 변수 설명] 
- format:day = 일자 데이터 (e.g. 1 = 01/01)
- hour = 시간(2자리) + 분(2자리) (e.g. 200 = 2:00 AM), UTC 기준
- forecast = hour를 기준으로 하여 몇 시간 뒤를 예측했는지
- value = 해당 시간에 대해 예측된 최저기온 값

[전처리 과정 설명] 
2:00(UTC시간대 기준)의 7시간 뒤의 예측값만을 추출하여 데이터프레임으로 구성하고자 함 
- hour == 200, forecast == 7인 행만을 조건 아래 추출 
- 조건 색인 이후 불필요한 열인 hour, forecast를 drop
- 1~30의 int 형태로 나열되어 있는 format:day열을 날짜 형태로 변환 

[기타 데이터 관련 설명] 
- 제주시 및 서귀포시 기준 행정동은 네이버 날씨의 대표값을 따름 (천지동의 경우 네이버 날씨에서는 서귀동을 기준으로 하지만 냘씨 예보는 법정동을 기준으로 하여 그에 준응하는 천지동의 데이터를 사용함) 
- 각 일자에 대한 예측값은 UTC기준 2:00AM (한국 시간, 11:00AM)에 발표된 해당 날짜의 18:00(KST)에 대한 예측값을 사용함

source : 기상청 - 기상자료개방포털 (2021/01-2021/06은 (구)단기 예보 파일셋 활용, 2021/07-2021/12은 단기 예보 파일셋 활용)
https://data.kma.go.kr/data/rmt/rmtList.do?code=420&pgmNo=572

In [1]:
import pandas as pd
from glob import glob

# 제주시 아라동 최저기온

In [2]:
# glob()사용하여 제주시 상반기,하반기 최저기온 데이터 하나의 데이터프레임으로 불러오기
file_names = glob('../Dataset/raw_data/weather/minimum_temp/minimum_temp_j/*.csv')
minimum_temp_1 = pd.DataFrame() 


for file_name in file_names:
    temp = pd.read_csv(file_name, encoding='cp949')
    temp.rename(columns = {temp.columns[0]:'day', temp.columns[3]:'minimum temperature'}, inplace=True)

    minimum_temp_1 = pd.concat([minimum_temp_1, temp])



minimum_temp_1

,day,hour,forecast,minimum temperature
0,1,200.0,4.0,2.0
1,1,200.0,7.0,-1.0
2,1,500.0,4.0,2.0
3,1,500.0,7.0,-1.0
4,1,800.0,4.0,2.0
...,...,...,...,...
3837,31,1700.0,8.0,0.0
3838,31,2000.0,6.0,2.0
3839,31,2000.0,7.0,0.0
3840,31,2300.0,6.0,2.0


In [3]:
# 정보 확인
minimum_temp_1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6743 entries, 0 to 3841
Data columns (total 4 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   day                  6743 non-null   object 
 1   hour                 6556 non-null   float64
 2   forecast             6556 non-null   float64
 3   minimum temperature  6556 non-null   float64
dtypes: float64(3), object(1)
memory usage: 263.4+ KB


In [4]:
# 결측치 확인 
minimum_temp_1.isnull().sum()

day                      0
hour                   187
forecast               187
minimum temperature    187
dtype: int64

In [5]:
# 자료형이 다양하여 여러 조건으로 조건 색인하여 데이터프레임 재구성
minimum_temp_j = minimum_temp_1[(minimum_temp_1['hour'] == 200) & (minimum_temp_1['forecast'] == 7) \
    |(minimum_temp_1['hour'] == '200.0') & (minimum_temp_1['forecast'] == '7.0') \
        |(minimum_temp_1['hour'] == '0200') & (minimum_temp_1['forecast'] == '+7')]

minimum_temp_j

,day,hour,forecast,minimum temperature
1,1,200.0,7.0,-1.0
17,2,200.0,7.0,0.0
33,3,200.0,7.0,3.0
49,4,200.0,7.0,-1.0
65,5,200.0,7.0,-4.0
...,...,...,...,...
3739,27,200.0,7.0,3.0
3760,28,200.0,7.0,2.0
3781,29,200.0,7.0,0.0
3802,30,200.0,7.0,-1.0


In [6]:
# 누락 데이터 확인
minimum_temp_j['day'].value_counts()

 1     12
 15    12
 28    12
 27    12
 26    12
 25    12
 23    12
 22    12
 21    12
 20    12
 19    12
 18    12
 17    12
 2     12
 16    12
 14    12
 13    12
 12    12
 11    12
 10    12
 9     12
 8     12
 7     12
 6     12
 5     12
 4     12
 3     12
 24    11
 29    11
 30    11
 31     7
Name: day, dtype: int64

In [7]:
# 재구성한 데이터프레임 결측치 확인
minimum_temp_j.isnull().sum()

day                    0
hour                   0
forecast               0
minimum temperature    0
dtype: int64

In [8]:
# 재구성한 데이터프레임 인덱스 초기화
minimum_temp_j = minimum_temp_j.reset_index(drop=True)
minimum_temp_j

,day,hour,forecast,minimum temperature
0,1,200.0,7.0,-1.0
1,2,200.0,7.0,0.0
2,3,200.0,7.0,3.0
3,4,200.0,7.0,-1.0
4,5,200.0,7.0,-4.0
...,...,...,...,...
359,27,200.0,7.0,3.0
360,28,200.0,7.0,2.0
361,29,200.0,7.0,0.0
362,30,200.0,7.0,-1.0


In [9]:
# 누락된 값 추가하기 위해 index 확인
minimum_temp_j[350:365]

,day,hour,forecast,minimum temperature
350,17,200.0,7.0,4.0
351,18,200.0,7.0,3.0
352,19,200.0,7.0,7.0
353,20,200.0,7.0,3.0
354,21,200.0,7.0,3.0
355,22,200.0,7.0,4.0
356,23,200.0,7.0,-1.0
357,25,200.0,7.0,-1.0
358,26,200.0,7.0,1.0
359,27,200.0,7.0,3.0


In [10]:
# 누락된 값 추가하기
new_row = pd.DataFrame([[24, 200.0, 7.0, int(-3.0)]], columns = minimum_temp_j.columns)
minimum_temp_j = pd.concat([minimum_temp_j.iloc[:357], new_row, minimum_temp_j.iloc[357:]], ignore_index = True)

minimum_temp_j

,day,hour,forecast,minimum temperature
0,1,200.0,7.0,-1.0
1,2,200.0,7.0,0.0
2,3,200.0,7.0,3.0
3,4,200.0,7.0,-1.0
4,5,200.0,7.0,-4.0
...,...,...,...,...
360,27,200.0,7.0,3.0
361,28,200.0,7.0,2.0
362,29,200.0,7.0,0.0
363,30,200.0,7.0,-1.0


In [11]:
# 누락된 값 추가 확인
minimum_temp_j[350:365]

,day,hour,forecast,minimum temperature
350,17,200.0,7.0,4.0
351,18,200.0,7.0,3.0
352,19,200.0,7.0,7.0
353,20,200.0,7.0,3.0
354,21,200.0,7.0,3.0
355,22,200.0,7.0,4.0
356,23,200.0,7.0,-1.0
357,24,200.0,7.0,-3.0
358,25,200.0,7.0,-1.0
359,26,200.0,7.0,1.0


In [12]:
# 불필요한 열 제거 후 float -> int형으로 변경
minimum_temp_j = minimum_temp_j.drop(columns = {'day','hour','forecast'}).astype(int)


minimum_temp_j

,minimum temperature
0,-1
1,0
2,3
3,-1
4,-4
...,...
360,3
361,2
362,0
363,-1


# 서귀포시 천지동 최저기온

In [15]:
# glob()사용하여 제주시 상반기 하반기 최저기온 데이터 하나의 데이터프레임으로 불러오기
file_names = glob('../Dataset/raw_data/weather/minimum_temp/minimum_temp_s/*.csv')
minimum_temp_2 = pd.DataFrame() 


for file_name in file_names:
    temp = pd.read_csv(file_name)
    temp.rename(columns = {temp.columns[0]:'day', temp.columns[3]:'minimum temperature'}, inplace=True)

    minimum_temp_2 = pd.concat([minimum_temp_2, temp])


minimum_temp_2

,day,hour,forecast,minimum temperature
0,1,200.0,4.0,6.0
1,1,200.0,7.0,3.0
2,1,500.0,4.0,6.0
3,1,500.0,7.0,3.0
4,1,800.0,4.0,5.0
...,...,...,...,...
7722,31,1700,+8,6.000000
7723,31,2000,+6,6.000000
7724,31,2000,+7,5.000000
7725,31,2300,+6,6.000000


In [16]:
# 자료형이 다양하여 여러 조건으로 조건 색인하여 데이터프레임 재구성
minimum_temp_s = minimum_temp_2[(minimum_temp_2['hour'] == 200) & (minimum_temp_2['forecast'] == 7) \
    | (minimum_temp_2['hour'] == '200.0') & (minimum_temp_2['forecast'] == '7.0') \
        |(minimum_temp_2['hour'] == '0200') & (minimum_temp_2['forecast'] == '+7')]
minimum_temp_s

,day,hour,forecast,minimum temperature
1,1,200.0,7.0,3.0
17,2,200.0,7.0,5.0
33,3,200.0,7.0,7.0
49,4,200.0,7.0,3.0
65,5,200.0,7.0,1.0
...,...,...,...,...
7624,27,0200,+7,9.000000
7645,28,0200,+7,5.000000
7666,29,0200,+7,3.000000
7687,30,0200,+7,3.000000


In [17]:
# 정보 확인
minimum_temp_s.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 549 entries, 1 to 7708
Data columns (total 4 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   day                  549 non-null    object
 1   hour                 549 non-null    object
 2   forecast             549 non-null    object
 3   minimum temperature  549 non-null    object
dtypes: object(4)
memory usage: 21.4+ KB


In [18]:
# 누락 데이터 확인
minimum_temp_s['day'].value_counts()

 1     18
 2     18
 28    18
 27    18
 26    18
 25    18
 24    18
 23    18
 22    18
 21    18
 20    18
 19    18
 18    18
 17    18
 16    18
 15    18
 14    18
 13    18
 12    18
 11    18
 10    18
 9     18
 8     18
 7     18
 6     18
 5     18
 4     18
 3     18
 29    17
 30    17
 31    11
Name: day, dtype: int64

In [19]:
# 누락된 데이터 추가
minimum_temp_s.loc[6619] = [31,200.0,7.0,7.0]

C:\Users\LeeSH\AppData\Local\Temp\ipykernel_20836\1411420610.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  minimum_temp_s.loc[6619] = [31,200.0,7.0,7.0]


In [20]:
# 데이터 추가 확인
minimum_temp_s

,day,hour,forecast,minimum temperature
1,1,200.0,7.0,3.0
17,2,200.0,7.0,5.0
33,3,200.0,7.0,7.0
49,4,200.0,7.0,3.0
65,5,200.0,7.0,1.0
...,...,...,...,...
7645,28,0200,+7,5.000000
7666,29,0200,+7,3.000000
7687,30,0200,+7,3.000000
7708,31,0200,+7,7.000000


In [21]:
# int형으로 변경 
minimum_temp_s['hour'].astype(int)
minimum_temp_s['forecast'].astype(int)

1       7
17      7
33      7
49      7
65      7
       ..
7645    7
7666    7
7687    7
7708    7
6619    7
Name: forecast, Length: 550, dtype: int32

In [22]:
# 자료형 변경 확인
minimum_temp_s

,day,hour,forecast,minimum temperature
1,1,200.0,7.0,3.0
17,2,200.0,7.0,5.0
33,3,200.0,7.0,7.0
49,4,200.0,7.0,3.0
65,5,200.0,7.0,1.0
...,...,...,...,...
7645,28,0200,+7,5.000000
7666,29,0200,+7,3.000000
7687,30,0200,+7,3.000000
7708,31,0200,+7,7.000000


In [23]:
# 불필요한 열 제거
minimum_temp_s.drop(['day','hour', 'forecast'], axis=1,  inplace=True)
minimum_temp_s

C:\Users\LeeSH\AppData\Local\Temp\ipykernel_20836\4188544141.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  minimum_temp_s.drop(['day','hour', 'forecast'], axis=1,  inplace=True)


,minimum temperature
1,3.0
17,5.0
33,7.0
49,3.0
65,1.0
...,...
7645,5.000000
7666,3.000000
7687,3.000000
7708,7.000000


In [24]:
# join 키 값 부여
minimum_temp_s.reset_index(inplace=True)
minimum_temp_s

,index,minimum temperature
0,1,3.0
1,17,5.0
2,33,7.0
3,49,3.0
4,65,1.0
...,...,...
545,7645,5.000000
546,7666,3.000000
547,7687,3.000000
548,7708,7.000000


In [25]:
# 불필요한 열 제거
minimum_temp_s.drop(['index'], axis=1,  inplace=True)
minimum_temp_s

C:\Users\LeeSH\AppData\Local\Temp\ipykernel_20836\2954934704.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  minimum_temp_s.drop(['index'], axis=1,  inplace=True)


,minimum temperature
0,3.0
1,5.0
2,7.0
3,3.0
4,1.0
...,...
545,5.000000
546,3.000000
547,3.000000
548,7.000000
